In [110]:
# load json from file
import json
import pickle
import random
with open('./games/tripple.json') as f:
    data = json.load(f)
# print the data
print(data)


{'start': {'actions': {'up': [{'p': 1, 'next': {'rewards': [{'p': 1, 'value': 1}], 'actions': {'left': [{'p': 1, 'next': {'rewards': [{'p': 1, 'value': 2}], 'actions': {'left': [{'p': 1, 'next': {'rewards': [{'p': 0.5, 'value': -1}, {'p': 0.5, 'value': -2}], 'actions': {'up': [{'p': 1, 'next': {'rewards': [{'p': 1, 'value': 1}]}}]}}}], 'down': [{'p': 1, 'next': {'rewards': [{'p': 1, 'value': 1}], 'actions': {'left': [{'p': 1, 'next': {'rewards': [{'p': 1, 'value': -5}], 'actions': {'up': [{'p': 1, 'next': {'rewards': [{'p': 0.5, 'value': -1}, {'p': 0.5, 'value': -2}], 'actions': {'up': [{'p': 1, 'next': {'rewards': [{'p': 1, 'value': 1}]}}]}}}]}}}]}}}]}}}], 'up': [{'p': 1, 'next': {'rewards': [{'p': 1, 'value': 1}], 'actions': {'left': [{'p': 1, 'next': {'rewards': [{'p': 1, 'value': 5}], 'actions': {'left': [{'p': 1, 'next': {'rewards': [{'p': 1, 'value': 1}]}}], 'down': [{'p': 1, 'next': {'rewards': [{'p': 1, 'value': 2}], 'actions': {'left': [{'p': 1, 'next': {'rewards': [{'p': 0.5,

In [98]:
class Node:
  def __init__(self, node, nodes):
    self.id = pickle.dumps(node)
    self.rewardHistory = []
    if 'rewards' in node:
      self.rewards = node['rewards']
    if 'actions' not in node:
      self.isLeaf = True
      return
    self.isLeaf = False
    if 'up' in node['actions']:
      self.up = getNode(node['actions']['up'][0]['next'], nodes)
    if 'down' in node['actions']:
      self.down = getNode(node['actions']['down'][0]['next'], nodes)
    if 'left' in node['actions']:
      self.left = getNode(node['actions']['left'][0]['next'], nodes)
    if 'right' in node['actions']:
      self.right = getNode(node['actions']['right'][0]['next'], nodes)

  def __str__(self):
    if self.isLeaf:
      return 'Leaf'
    return 'Node'
  
  def getReward(self):
    # get random number
    r = random.random()
    # sample reward
    s = 0
    if not hasattr(self, 'rewards'):
      return 0
    for i in range(len(self.rewards)):
      s += self.rewards[i]['p']
      if r < s:
        reward = self.rewards[i]['value']
        self.rewardHistory.append(reward)
        return reward
      
  def getAverageReward(self):
    return sum(self.rewardHistory) / len(self.rewardHistory) if len(self.rewardHistory) > 0 else 0

def getNode(node, nodes):
  # return node in nodes with matching id
  for n in nodes:
    if n.id == pickle.dumps(node):
      return n
  node = Node(node, nodes)
  nodes.append(node)
  return node


In [99]:
nodes = []
start = Node(data['start'], nodes)


In [112]:
class Path:
  def __init__(self, path):
    self.path = path
    self.length = len(path)

def getPaths(node, path):
  if node.isLeaf:
    return [Path(path + [node])]
  paths = []
  if hasattr(node, 'left'):
    paths += getPaths(node.left, path + [node])
  if hasattr(node, 'up'):
    paths += getPaths(node.up, path + [node])
  if hasattr(node, 'down'):
    paths += getPaths(node.down, path + [node])
  if hasattr(node, 'right'):
    paths += getPaths(node.right, path + [node])
  return paths

In [107]:
paths = getPaths(start, [])
print(len(paths))

2


In [113]:
def selectBestPath(paths):
  # select path with highest average reward
  maxAverageReward = 0
  maxPath = None
  rewards = []
  for path in paths:
    reward = 0
    for node in path.path:
      reward += node.getAverageReward()
    rewards.append(reward)
  # get max from rewards
  bestPath = max(range(len(rewards)), key=rewards.__getitem__)
  return paths[bestPath]

def epsilonGreedy(selectBestPath, epsilon):
  def selectPath(paths):
    # select path with highest average reward
    if random.random() < epsilon:
      return random.choice(paths)
    return selectBestPath(paths)
  return selectPath


In [117]:
for epsilon in range(0, 100, 5):
    scores = []
    for i in range(10):
        nodes = []
        start = Node(data['start'], nodes)
        paths = getPaths(start, [])

        selectPath = epsilonGreedy(selectBestPath, epsilon/100.)

        score = 0
        for i in range(75):
            # select path
            path = selectPath(paths)
            # traverse path to update reward
            reward = 0
            for node in path.path:
                reward += node.getReward()
        #    print('Reward: ' + str(reward))
            score += reward
        scores.append(score)

    avgScore = sum(scores) / len(scores)
    print(str(epsilon) + ' Score: ' + str(avgScore))

0 Score: 587.8
5 Score: 592.3
10 Score: 578.4
15 Score: 555.5
20 Score: 544.0
25 Score: 531.6
30 Score: 481.3
35 Score: 482.7
40 Score: 446.9
45 Score: 443.4
50 Score: 408.4
55 Score: 401.4
60 Score: 355.7
65 Score: 358.5
70 Score: 330.5
75 Score: 324.3
80 Score: 300.8
85 Score: 282.8
90 Score: 259.0
95 Score: 227.8


In [104]:
# print average reward for each node
for node in nodes:
  print(node.getAverageReward())
  

0
0


In [105]:
# print lenth of average reward history for each node
for node in nodes:
  print(len(node.rewardHistory))
  

0
0
